In [21]:
# setting up the libraries that we will need 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, cross_val_score
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [22]:
# loading the data
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
train.name = "Train"
test.name = "Test"

In [23]:
# def corr(df):
#     corr_matrix = df.corr()
#     print(corr_matrix["label"])

In [24]:
#data exploration
train.head()

,Unnamed: 0,time,lat,sid,label,lon,Unnamed: 0.1,lid,t_user_id,altitude,td,distance,speed,acc,bearing,jerk,brate,brrate,trial_railway_shortest_distance,distance_log
0,0,2007-04-29 15:34:24,39.973833,0,bike,116.332967,713361,640,21,328.08399,8.0,21.225941,2.653243,-0.189673,290.447082,0.023643,-19.508054,4.003895,2161.538565,1.326867
1,1,2007-04-29 15:34:32,39.973900,0,bike,116.332733,713362,640,21,328.08399,14.0,15.902029,1.135859,-0.000529,134.382649,0.019251,12.523108,-1.397089,2177.166846,1.201453
2,2,2007-04-29 15:34:46,39.973800,0,bike,116.332867,713363,640,21,328.08399,18.0,20.312142,1.128452,0.268987,309.706160,-0.016452,-7.036139,0.446452,2162.100936,1.307756
3,3,2007-04-29 15:35:04,39.973917,0,bike,116.332683,713364,640,21,328.08399,125.0,746.277432,5.970219,-0.027141,183.055660,-0.000987,1.000006,0.002074,2180.763143,2.872900
4,4,2007-04-29 15:37:09,39.967217,0,bike,116.332217,713365,640,21,328.08399,7.0,18.043469,2.577638,-0.150516,308.056352,0.037949,1.259286,-1.943406,1582.935115,1.256320


In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       972 non-null    int64  
 1   time                             972 non-null    object 
 2   lat                              972 non-null    float64
 3   sid                              972 non-null    int64  
 4   label                            972 non-null    object 
 5   lon                              972 non-null    float64
 6   Unnamed: 0.1                     972 non-null    int64  
 7   lid                              972 non-null    int64  
 8   t_user_id                        972 non-null    int64  
 9   altitude                         972 non-null    float64
 10  td                               972 non-null    float64
 11  distance                         972 non-null    float64
 12  speed                 

In [26]:
train.describe()


,Unnamed: 0,lat,sid,lon,Unnamed: 0.1,lid,t_user_id,altitude,td,distance,speed,acc,bearing,jerk,brate,brrate,trial_railway_shortest_distance,distance_log
count,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.0,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,9.720000e+02,972.000000
mean,485.500000,31.223116,0.166667,116.445687,714346.888889,647.770576,21.0,1213.633982,412.757202,2074.849504,12.339225,-0.049772,172.893953,0.026876,-0.367543,0.266131,8.358144e+05,2.239201
std,280.736531,3.945312,0.553055,1.338575,713.848145,6.016441,0.0,1882.034934,3813.366599,9038.926790,25.716790,2.088298,82.226553,0.447048,9.841408,4.784026,4.026103e+05,0.925230
min,0.000000,28.042367,0.000000,113.794933,713361.000000,640.000000,21.0,0.000000,2.000000,0.000000,0.000000,-17.755512,0.000000,-5.815833,-157.864232,-34.853519,5.360161e+00,0.000000
25%,242.750000,28.389071,0.000000,115.680654,713714.750000,643.000000,21.0,167.322835,20.000000,24.745137,0.796439,-0.028523,119.883024,-0.001020,-0.375743,-0.017068,7.316769e+05,1.393489
50%,485.500000,29.452525,0.000000,116.818275,714160.500000,647.000000,21.0,219.816273,38.500000,117.679985,3.679879,0.000050,176.157678,0.000005,-0.000579,-0.000007,1.013410e+06,2.070688
75%,728.250000,32.259250,0.000000,117.976796,715055.250000,655.000000,21.0,1126.148294,111.250000,963.552528,24.873571,0.013408,215.059660,0.001235,0.412344,0.018026,1.129738e+06,2.983875
max,971.000000,39.973917,2.000000,118.063683,715743.000000,657.000000,21.0,5347.769029,74763.000000,137847.550126,698.878256,57.088406,359.687272,8.401501,69.197754,84.147085,1.164705e+06,5.139399


In [27]:
# corr(train)


In [28]:
test.describe()


,Unnamed: 0,lat,lon,Unnamed: 0.1,lid,t_user_id,altitude,td,distance,speed,acc,bearing,jerk,brate,brrate,distance_log
count,972.000000,972.000000,972.000000,972.000000,972.000000,972.0,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000,972.000000
mean,485.500000,31.223116,116.445687,714346.888889,647.770576,21.0,1213.633982,412.757202,2074.849504,12.339225,-0.049772,172.893953,0.026876,-0.367543,0.266131,2.239201
std,280.736531,3.945312,1.338575,713.848145,6.016441,0.0,1882.034934,3813.366599,9038.926790,25.716790,2.088298,82.226553,0.447048,9.841408,4.784026,0.925230
min,0.000000,28.042367,113.794933,713361.000000,640.000000,21.0,0.000000,2.000000,0.000000,0.000000,-17.755512,0.000000,-5.815833,-157.864232,-34.853519,0.000000
25%,242.750000,28.389071,115.680654,713714.750000,643.000000,21.0,167.322835,20.000000,24.745137,0.796439,-0.028523,119.883024,-0.001020,-0.375743,-0.017068,1.393489
50%,485.500000,29.452525,116.818275,714160.500000,647.000000,21.0,219.816273,38.500000,117.679985,3.679879,0.000050,176.157678,0.000005,-0.000579,-0.000007,2.070688
75%,728.250000,32.259250,117.976796,715055.250000,655.000000,21.0,1126.148294,111.250000,963.552528,24.873571,0.013408,215.059660,0.001235,0.412344,0.018026,2.983875
max,971.000000,39.973917,118.063683,715743.000000,657.000000,21.0,5347.769029,74763.000000,137847.550126,698.878256,57.088406,359.687272,8.401501,69.197754,84.147085,5.139399


In [ ]:
coordinates_columns = ['distance', 'speed', 'acc', 'bearing', 'jerk']
data_set = [train, test]
for coord in coordinates_columns:
    for data in data_set:
        maxi = data[coord].max()
        mini = data[coord].min()
        print ("Range of {} in {} data is : ({:.3f}, {:.3f})".format(coord, data.name, maxi, mini))
del data_set